In [1]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt

import qcodes as qc
from qcodes import ParamSpec, new_data_set
from qcodes.dataset.database import initialise_database
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import load_or_create_experiment

from plottr.data.qcodes_dataset import datadict_from_path_and_run_id

In [2]:
DBPATH = './test_qc_saveandload.db'

# Make 2D softsweep data

## define sweep

In [7]:
qc.config.core.db_location = DBPATH
initialise_database()
exp = load_or_create_experiment('2d_softsweep', sample_name='no sample')

x = np.linspace(0, 1., 11)
y = np.linspace(0, 1., 11)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.random.rand(*xx.shape)

def get_results():
    for x, y, z in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)):
        yield dict(x=x, y=y, z=z)

## dummy measurement and retrieve as ddict

In [5]:
ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                  specs=[ParamSpec('x', 'numeric', unit='A'),
                         ParamSpec('y', 'numeric', unit='B'),
                         ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

results = get_results()
for r in results:
    ds.add_result(r)
ds.mark_complete()

run_id = ds.run_id
ddict = datadict_from_path_and_run_id(DBPATH, run_id)

## Manual insertion and retrieval of updates

In [9]:
_ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                  specs=[ParamSpec('x', 'numeric', unit='A'),
                         ParamSpec('y', 'numeric', unit='B'),
                         ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

run_id = _ds.run_id
results = get_next_result()

In [15]:
_ds.add_result(next(results))

ds = DataSet(path_to_db=DBPATH, run_id=run_id)
ds.number_of_results

2

In [20]:
ds.get_data('x', 'y', 'z', start=2)

[[0, 0.1, 0.769212716898808]]

In [17]:
ds.get_data?

ImportError: cannot import name 'NoneType'